In [1]:
import os
from pprint import pprint
import pathlib
import pandas as pd
from collections import defaultdict

data_dirs = "/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data"

# Define the mapping dictionary
NAME_MAPPING = {
    "บ้านท่าสี": "banthasi",
    "บ้านสบป้าด": "bansoppad",
    "บ้านสบเมาะ": "bansopmoe",
    "บ้านหัวฝาย": "banhuafai",
    "บ้านหางฮุง": "banhanghung",
}

# get all csv files in data_dirs
csv_files = list(pathlib.Path(data_dirs).rglob("*.csv"))
csv_files

[PosixPath('/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Banhanghung.csv'),
 PosixPath('/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Banhuafai.csv'),
 PosixPath('/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Banhuaiking.csv'),
 PosixPath('/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Banmaechang.csv'),
 PosixPath('/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Banmairattakosin.csv'),
 PosixPath('/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Bansadej.csv'),
 PosixPath('/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Bansopmoe.csv'),
 PosixPath('/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Bansoppad.csv'),
 PosixPath('/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Banthasii.csv'),
 PosixPath('/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Government center.csv'),
 PosixPath('/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2

In [2]:
NAME_MAPPING = {
    "บ้านท่าสี": {"name_eng": "banthasi", "name_th": "บ้านท่าสี"},
    "บ้านสบป้าด": {"name_eng": "bansoppad", "name_th": "บ้านสบป้าด"},
    "บ้านสบเมาะ": {"name_eng": "bansopmoe", "name_th": "บ้านสบเมาะ"},
    "บ้านหัวฝาย": {"name_eng": "banhuafai", "name_th": "บ้านหัวฝาย"},
    "บ้านหางฮุง": {"name_eng": "banhanghung", "name_th": "บ้านหางฮุง"},
    "บ้านห้วยคิง": {"name_eng": "banhuaiking", "name_th": "บ้านห้วยคิง"},
    "บ้านเสด็จ": {"name_eng": "bansadej", "name_th": "บ้านเสด็จ"},
    "บ้านแม่จาง": {"name_eng": "banmaechang", "name_th": "บ้านแม่จาง"},
    "บ้านใหม่รัตนโกสินทร์": {"name_eng": "banmairattanakosin", "name_th": "บ้านใหม่รัตนโกสินทร์"},
    "ประตูผา": {"name_eng": "pratupha", "name_th": "ประตูผา"},
    "ศูนย์ราชการแม่เมาะ": {"name_eng": "governmentcenter", "name_th": "ศูนย์ราชการแม่เมาะ"},
    "สถานีตรวจอากาศหลัก": {"name_eng": "mainstation", "name_th": "สถานีตรวจอากาศหลัก"},
    "สถานีหลัก": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    # เพิ่ม mapping ใหม่จาก unmapped files
    "บ้านห้วยฝาย": {
        "name_eng": "banhuafai",
        "name_th": "บ้านห้วยฝาย",
    },  # ชื่ออาจซ้ำกับ บ้านหัวฝาย
    "Banmairattakosin": {
        "name_eng": "banmairattanakosin",
        "name_th": "บ้านใหม่รัตนโกสินทร์",
    },  # การสะกดผิด
    "Banthasii": {"name_eng": "banthasi", "name_th": "บ้านท่าสี"},  # การสะกดผิด
    "Government center": {"name_eng": "governmentcenter", "name_th": "ศูนย์ราชการแม่เมาะ"},
    "Main station": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "Banhuayking": {"name_eng": "banhuaiking", "name_th": "บ้านห้วยคิง"},  # การสะกดผิด
    "BanHuayking": {"name_eng": "banhuaiking", "name_th": "บ้านห้วยคิง"},  # การสะกดผิด
    "Maechang": {"name_eng": "banmaechang", "name_th": "บ้านแม่จาง"},  # ชื่อย่อ
    # ชื่อที่มี suffix พิเศษ
    "บ้านท่าสี station": {"name_eng": "banthasi", "name_th": "บ้านท่าสี"},
    "บ้านสบป้าด station": {"name_eng": "bansoppad", "name_th": "บ้านสบป้าด"},
    "บ้านหางฮุง station": {"name_eng": "banhanghung", "name_th": "บ้านหางฮุง"},
    "บ้านใหม่รัตนโกสินทร์ station": {
        "name_eng": "banmairattanakosin",
        "name_th": "บ้านใหม่รัตนโกสินทร์",
    },
    "ประตูผา station": {"name_eng": "pratupha", "name_th": "ประตูผา"},
    "ศูนย์ราชการแม่เมาะ station": {
        "name_eng": "governmentcenter",
        "name_th": "ศูนย์ราชการแม่เมาะ",
    },
    # Main station variations
    "Main station WDH": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "Main station WDL": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "Main station WDM": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "Main station WSHxls": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "Main station WSLxls": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "Main station WSMxls": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "สถานีหลัก-ไม่มีข้อมูล": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
}

In [3]:
data_dirs = "/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data"
csv_files = list(pathlib.Path(data_dirs).rglob("*.csv"))

NAME_MAPPING = {
    "บ้านท่าสี": {"name_eng": "banthasi", "name_th": "บ้านท่าสี"},
    "บ้านสบป้าด": {"name_eng": "bansoppad", "name_th": "บ้านสบป้าด"},
    "บ้านสบเมาะ": {"name_eng": "bansopmoe", "name_th": "บ้านสบเมาะ"},
    "บ้านหัวฝาย": {"name_eng": "banhuafai", "name_th": "บ้านหัวฝาย"},
    "บ้านหางฮุง": {"name_eng": "banhanghung", "name_th": "บ้านหางฮุง"},
    "บ้านห้วยคิง": {"name_eng": "banhuaiking", "name_th": "บ้านห้วยคิง"},
    "บ้านเสด็จ": {"name_eng": "bansadej", "name_th": "บ้านเสด็จ"},
    "บ้านแม่จาง": {"name_eng": "banmaechang", "name_th": "บ้านแม่จาง"},
    "บ้านใหม่รัตนโกสินทร์": {"name_eng": "banmairattanakosin", "name_th": "บ้านใหม่รัตนโกสินทร์"},
    "ประตูผา": {"name_eng": "pratupha", "name_th": "ประตูผา"},
    "ศูนย์ราชการแม่เมาะ": {"name_eng": "governmentcenter", "name_th": "ศูนย์ราชการแม่เมาะ"},
    "สถานีตรวจอากาศหลัก": {"name_eng": "mainstation", "name_th": "สถานีตรวจอากาศหลัก"},
    "สถานีหลัก": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "บ้านห้วยฝาย": {"name_eng": "banhuafai", "name_th": "บ้านห้วยฝาย"},
    "Banmairattakosin": {
        "name_eng": "banmairattanakosin",
        "name_th": "บ้านใหม่รัตนโกสินทร์",
    },
    "Banthasii": {"name_eng": "banthasi", "name_th": "บ้านท่าสี"},
    "Government center": {"name_eng": "governmentcenter", "name_th": "ศูนย์ราชการแม่เมาะ"},
    "Main station": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "Banhuayking": {"name_eng": "banhuaiking", "name_th": "บ้านห้วยคิง"},
    "BanHuayking": {"name_eng": "banhuaiking", "name_th": "บ้านห้วยคิง"},
    "Maechang": {"name_eng": "banmaechang", "name_th": "บ้านแม่จาง"},
    "บ้านท่าสี station": {"name_eng": "banthasi", "name_th": "บ้านท่าสี"},
    "บ้านสบป้าด station": {"name_eng": "bansoppad", "name_th": "บ้านสบป้าด"},
    "บ้านหางฮุง station": {"name_eng": "banhanghung", "name_th": "บ้านหางฮุง"},
    "บ้านใหม่รัตนโกสินทร์ station": {
        "name_eng": "banmairattanakosin",
        "name_th": "บ้านใหม่รัตนโกสินทร์",
    },
    "ประตูผา station": {"name_eng": "pratupha", "name_th": "ประตูผา"},
    "ศูนย์ราชการแม่เมาะ station": {
        "name_eng": "governmentcenter",
        "name_th": "ศูนย์ราชการแม่เมาะ",
    },
    "Main station WDH": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "Main station WDL": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "Main station WDM": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "Main station WSHxls": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "Main station WSLxls": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "Main station WSMxls": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
    "สถานีหลัก-ไม่มีข้อมูล": {"name_eng": "mainstation", "name_th": "สถานีหลัก"},
}


def parse_filename(filepath):
    filename = pathlib.Path(filepath).name
    # replace _
    filename = filename.replace("_", " ")
    parts = filename.split("#")

    if len(parts) != 2:
        return None, None

    # ใช้ทุกอย่างก่อน # เป็น class_type (รวมช่องว่างและขีดใต้)
    first_part = parts[0].split(" ")

    class_type = first_part[0].lower()  # เช่น "pm10 2009", "so2_2009-2010"
    name_part = parts[1].replace(".csv", "")

    return class_type, name_part


normalized_list = []
unmapped_files = []

for file_path in csv_files:
    class_type, name_part = parse_filename(str(file_path))
    if not class_type or not name_part:
        unmapped_files.append(
            {"file": str(file_path), "reason": "Invalid filename format"}
        )
        continue

    mapped_entry = None
    is_mapped = False

    for thai_name, mapping in NAME_MAPPING.items():
        if (
            thai_name == name_part
            or mapping["name_eng"].lower() == name_part.lower()
            or mapping["name_th"] == name_part
        ):
            mapped_entry = mapping
            is_mapped = True
            break

    if mapped_entry:
        normalized_entry = {
            "original": str(file_path),
            "class": class_type,  # เช่น "pm10 2009", "so2_2009-2010"
            "name_th": mapped_entry["name_th"],
            "name_en": mapped_entry["name_eng"],
        }
    else:
        normalized_entry = {
            "original": str(file_path),
            "class": class_type,
            "name_th": name_part if name_part.startswith("บ้าน") else "",
            "name_en": name_part.lower(),
        }
        unmapped_files.append(
            {
                "file": str(file_path),
                "reason": f"No mapping found for name: {name_part}",
            }
        )

    normalized_list.append(normalized_entry)

# Print results
print("Normalized List (first 5 entries):")
pprint(normalized_list[:])

print("\nUnmapped Files:")
if unmapped_files:
    for unmapped in unmapped_files:
        print(f"File: {unmapped['file']}")
        print(f"Reason: {unmapped['reason']}\n")
else:
    print("All files successfully mapped!")

print(f"\nSummary:")
print(f"Total files processed: {len(csv_files)}")
print(f"Successfully mapped: {len(normalized_list)}")
print(f"Unmapped files: {len(unmapped_files)}")

Normalized List (first 5 entries):
[{'class': 'bp',
  'name_en': 'banhanghung',
  'name_th': 'บ้านหางฮุง',
  'original': '/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Banhanghung.csv'},
 {'class': 'bp',
  'name_en': 'banhuafai',
  'name_th': 'บ้านหัวฝาย',
  'original': '/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Banhuafai.csv'},
 {'class': 'bp',
  'name_en': 'banhuaiking',
  'name_th': 'บ้านห้วยคิง',
  'original': '/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Banhuaiking.csv'},
 {'class': 'bp',
  'name_en': 'banmaechang',
  'name_th': 'บ้านแม่จาง',
  'original': '/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Banmaechang.csv'},
 {'class': 'bp',
  'name_en': 'banmairattanakosin',
  'name_th': 'บ้านใหม่รัตนโกสินทร์',
  'original': '/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Banmairattakosin.csv'},
 {'class': 'bp',
  'name_en': 'bansadej',
  'name_th': 'บ้านเสด็จ',
  'original': '/mnt/e/MayThesis2025/sr

In [4]:
# Output folder
output_dir = "/mnt/e/MayThesis2025/src/labs4/v02_merge/output"
os.makedirs(output_dir, exist_ok=True)

# Group by station name_en
station_files = defaultdict(list)
for entry in normalized_list:
    station_files[entry["name_en"]].append(entry)

# Process each station separately
for station, files in station_files.items():
    # Initialize empty merged DataFrame for this station
    merged_df = pd.DataFrame()

    # Process each file for the station
    for file_info in files:
        file_path = file_info["original"]
        data_type = file_info["class"]

        # Load data
        try:
            df = pd.read_csv(file_path, parse_dates=["Timestamp"])
        except Exception as e:
            print(f"[ERROR] reading {file_path}: {e}")
            continue

        df.rename(columns={"Value": data_type}, inplace=True)

        # Check for duplicated timestamps in current file
        duplicated_current = df[df.duplicated(subset=["Timestamp"], keep=False)]
        if not duplicated_current.empty:
            print(f"[WARNING] Duplicate timestamps in file '{file_path}':")
            print(duplicated_current)
            continue  # You might skip or handle differently as per your choice

        # Merge to existing merged_df DataFrame
        if merged_df.empty:
            merged_df = df.copy()
        else:
            if data_type in merged_df.columns:
                # Update existing column values where timestamps match
                merged_df = merged_df.merge(
                    df[["Timestamp", data_type]],
                    on="Timestamp",
                    how="outer",
                    suffixes=("", "_new"),
                )

                # If there are new data (non-NA), update original column
                merged_df[data_type] = merged_df[data_type].combine_first(
                    merged_df[f"{data_type}_new"]
                )
                merged_df.drop(columns=[f"{data_type}_new"], inplace=True)
            else:
                # Add new column to merged_df
                merged_df = merged_df.merge(df, on="Timestamp", how="outer")

    # Sort merged data by timestamp
    merged_df.sort_values("Timestamp", inplace=True)

    # Final check for duplicated timestamps after merging
    final_dup = merged_df[merged_df.duplicated(subset=["Timestamp"], keep=False)]

    # rename Timestamp to Datetime
    merged_df.rename(columns={"Timestamp": "Datetime"}, inplace=True)
    if not final_dup.empty:
        print(
            f"[ERROR] Final merged DataFrame for '{station}' has duplicated timestamps:"
        )
        print(final_dup)

    # Save merged DataFrame
    output_file = os.path.join(output_dir, f"{station}.csv")
    merged_df.to_csv(output_file, index=False)
    print(f"[SUCCESS] Merged file created: {output_file}")

[SUCCESS] Merged file created: /mnt/e/MayThesis2025/src/labs4/v02_merge/output/banhanghung.csv
[SUCCESS] Merged file created: /mnt/e/MayThesis2025/src/labs4/v02_merge/output/banhuafai.csv
[SUCCESS] Merged file created: /mnt/e/MayThesis2025/src/labs4/v02_merge/output/banhuaiking.csv
[SUCCESS] Merged file created: /mnt/e/MayThesis2025/src/labs4/v02_merge/output/banmaechang.csv
[SUCCESS] Merged file created: /mnt/e/MayThesis2025/src/labs4/v02_merge/output/banmairattanakosin.csv
[SUCCESS] Merged file created: /mnt/e/MayThesis2025/src/labs4/v02_merge/output/bansadej.csv
[SUCCESS] Merged file created: /mnt/e/MayThesis2025/src/labs4/v02_merge/output/bansopmoe.csv
[SUCCESS] Merged file created: /mnt/e/MayThesis2025/src/labs4/v02_merge/output/bansoppad.csv
[SUCCESS] Merged file created: /mnt/e/MayThesis2025/src/labs4/v02_merge/output/banthasi.csv
[SUCCESS] Merged file created: /mnt/e/MayThesis2025/src/labs4/v02_merge/output/governmentcenter.csv
[SUCCESS] Merged file created: /mnt/e/MayThesis202

In [5]:
station_files

defaultdict(list,
            {'banhanghung': [{'original': '/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2009#Banhanghung.csv',
               'class': 'bp',
               'name_th': 'บ้านหางฮุง',
               'name_en': 'banhanghung'},
              {'original': '/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/BP_2010-2020#บ้านหางฮุง.csv',
               'class': 'bp',
               'name_th': 'บ้านหางฮุง',
               'name_en': 'banhanghung'},
              {'original': '/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/DEV_2010-2020#บ้านหางฮุง.csv',
               'class': 'dev',
               'name_th': 'บ้านหางฮุง',
               'name_en': 'banhanghung'},
              {'original': '/mnt/e/MayThesis2025/src/labs4/v02_merge/cleaned_data/RAIN_2010-2020#บ้านหางฮุง.csv',
               'class': 'rain',
               'name_th': 'บ้านหางฮุง',
               'name_en': 'banhanghung'},
              {'original': '/mnt/e/MayThesis2025/src/labs4/v02_merge/